In [1]:
!ls

 cf_model.py	  q.py
 config.py	  README.md
 dataset.py	 'Spider-0_doing_Spider-Man: Far From Home.txt'
 db.ipynb	 'Spider-1_doing_Spider-Man: Far From Home.txt'
 done.txt	 'Spider-4_doing_Spider-Man: Far From Home.txt'
 fdstests.ipynb   todo.txt
 filemanager.py   todo.txt.lock
 letterboxd.db	  train.ipynb
 main.ipynb	  user.py
 menv		  wandb
 movie.py	  web_spider.py
 __pycache__


In [2]:
# Import the IMDb class from the imdb package
from imdb import IMDb
import requests
import time

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # Import the By class
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import import_ipynb
from user import User
from movie import Movie
import db

from faker import Faker
import random

from config import TMDB_API_KEY
from imdb import Cinemagoer
from collections import deque

from datetime import datetime

from filemanager import FileManager
from q import Queue
from web_spider import WebSpider

from googletrans import Translator, LANGUAGES

import threading


import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

import string


translator = Translator()

print(TMDB_API_KEY)  # Just to verify it's imported correctly; remove in production

month_map = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
             "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
import uuid

importing Jupyter notebook from db.ipynb
 cf_model.py	  q.py
 config.py	  README.md
 dataset.py	 'Spider-0_doing_Spider-Man: Far From Home.txt'
 db.ipynb	 'Spider-1_doing_Spider-Man: Far From Home.txt'
 done.txt	 'Spider-4_doing_Spider-Man: Far From Home.txt'
 fdstests.ipynb   todo.txt
 filemanager.py   todo.txt.lock
 letterboxd.db	  train.ipynb
 main.ipynb	  user.py
 menv		  wandb
 movie.py	  web_spider.py
 __pycache__
e47716bb6a8a0524a757f0b9320f0b29


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/james/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
#common practise to have users rate at least 20-30 reviews each....
TOTAL_MOVIES = 30
TOTAL_USERS = 2400
REVIEW_PAGES = 300

#movie['id'],movie['title'],movie['release_date'],movie['director_id'],movie['overview']
#mr['movie_id'],mr['user_id'],mr['content'],mr['ratings'],mr['date']
# movie['id'],movie['title'],movie['release_date'],movie['director_id'],movie['overview'])

In [4]:
def fetch_movie_names(api_key=TMDB_API_KEY, total_movies=10):
    
    base_url = "https://api.themoviedb.org/3/discover/movie"
    movie_count = 0
    page = 1
    while movie_count < total_movies:
        params = {
            "api_key": api_key,
            "language": "en-US",
            "sort_by": "revenue.desc",
            "page": page
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            for movie in data['results']:

                movie_count += 1
                yield movie['title']
                if movie_count >= total_movies:
                    break
        else:
            print(f"Failed to fetch movies: {response.status_code}")
            break
        page += 1
        time.sleep(1)  # Respect the rate limit
        
    # movie_names[:total_movies]



In [5]:
def scrape_letterboxd_movie(movie_name,review_pages_count):

    # Specify the path to chromedriver
    service = Service(executable_path='/home/james/.wdm/drivers/chromedriver/linux64/121.0.6167.85/chromedriver-linux64/chromedriver')
    # Create ChromeOptions object
    chrome_options = webdriver.ChromeOptions()
    # Disable popups
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')  # Disable GPU accelerati
    chrome_options.add_argument("--disable-popup-blocking")

    # Initialize the WebDriver with the Service object
    driver = webdriver.Chrome(service=service,options = chrome_options)

    # Wait for the page to load (optional, could use more sophisticated waits)
    
    driver.get('https://letterboxd.com/film/'+ movie_name.replace(' ','-').lower()+ '/')

    # Wait for the cookies consent button to be clickable, and then click it
    wait = WebDriverWait(driver, 30)

    movie = {'title':movie_name,'id':str(uuid.uuid4())}  # Initialize the dictionary once, outside the loop
    
    try:
        
        tabbed_content_blocks = driver.find_elements(By.XPATH, "//div[contains(@class, 'tabbed-content-block')]")
    
        # Loop through each 'tabbed-content' div
        for tcb_e in tabbed_content_blocks:
            name = tcb_e.get_attribute('id').split('-')[-1]
            if name == 'cast':
                movie[name] = [ae.get_attribute('textContent') for ae in tcb_e.find_elements(By.TAG_NAME, 'a')]
            elif name == 'crew':
                movie['director'] = [e.get_attribute('innerText') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/director/')]")]
                movie['director id'] = str(uuid.uuid4())
                movie['writer'] = [e.get_attribute('innerText') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/writer/')]")]
            elif name == 'details':
                movie['country'] = [e.get_attribute('textContent') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/films/country/')]")][0]
            elif name == 'genres':
                movie['genres'] = [e.get_attribute('textContent') for e in tcb_e.find_elements(By.XPATH, ".//a[starts-with(@href, '/films/genre/')]")]
            elif name == 'releases':
                movie['release date'] = tcb_e.find_element(By.XPATH, ".//h5[contains(@class, 'date')]").get_attribute('textContent')
        # reviews
        # Wait for the page to load (optional, could use more sophisticated waits)
        url = 'https://letterboxd.com/film/'+ movie_name.replace(' ','-').lower()+ '/reviews/'
        driver.get(url)

        wait = WebDriverWait(driver, 30)
        
        movie['reviews'] = []
        count = 0
        max_count = review_pages_count
        # and driver.find_element(By.XPATH, "//a[contains(@class, 'next')]"
        while count < max_count:

            review_divs = driver.find_elements(By.XPATH, "//div[contains(@class, 'film-detail-content')]")
        
            for rv in review_divs:
               
                rvw = {'movie_id':movie['id']}

                try:
                    rvw['ratings'] = rv.find_element(By.XPATH, ".//span[starts-with(@class, 'rating')]").get_attribute('class').split('-')[-1]
                    # print(rvw['ratings'])
                except:
                    #impute missing values later....
                    rvw['ratings'] = -1.0

                rvw['date'] = rv.find_element(By.XPATH, ".//span[starts-with(@class, 'date')]").get_attribute('textContent')
                rvw['content'] = rv.find_element(By.XPATH, ".//div[starts-with(@class, 'body-text')]").get_attribute('textContent')
                movie['reviews'].append(rvw)
            
            next_link = driver.find_element(By.XPATH, "//a[contains(@class, 'next')]")
            # next_button = driver.find_element(By.XPATH, "//a[contains(@class, 'next')]")
            driver.execute_script("arguments[0].click();", next_link)

            count += 1
    except Exception as e:
        print(movie_name)
        print(url)
        print(count)
    finally:
        driver.quit()
    
    return movie


In [6]:
def process_letterboxd_movie(movie):

    #TODO translate to language...
    # print(len(movie['reviews']))
    try:
        date = movie['release date'].strip().split()
        # print(date)
        movie['release date'] = datetime(int(date[2]),month_map[date[1]],int(date[0]))
    except Exception as e:
        print(movie)
    for movie_review in movie['reviews']:
        # print(movie_review['date'])
        date = movie_review['date'].strip().replace('  ',' ').split(' ')[-3:]
        # print(date)
        movie_review['date'] = datetime(int(date[2]),month_map[date[1]],int(date[0]))


            
        if movie_review['ratings'] == -1 and movie_review['content']:
            
            #do quick sentiment analysis.
            try:
                movie_review['content'] = translator.translate(movie_review['content'].strip(),dest='en').text
                
                sia = SentimentIntensityAnalyzer()
                score = sia.polarity_scores(movie_review['content'])
                sentiment = score['compound']  # Compound score
                movie_review['ratings'] = round(((sentiment + 1) / 2),2)
                print(movie_review['ratings'])
            except Exception as e:
                print(e)
                print(movie_review['content'])
                
        else:                
            
        
            movie_review['ratings'] = round(float(movie_review['ratings'])/10.0,2)

    return movie
        
    

In [7]:
mov = scrape_letterboxd_movie('star-wars',2)
process_letterboxd_movie(mov)

{'title': 'star-wars',
 'id': '83e9f40a-d103-4e7b-bb33-fb7f46eb9a90',
 'cast': ['Mark Hamill',
  'Harrison Ford',
  'Carrie Fisher',
  'Peter Cushing',
  'Alec Guinness',
  'Anthony Daniels',
  'Kenny Baker',
  'Peter Mayhew',
  'David Prowse',
  'James Earl Jones',
  'Phil Brown',
  'Shelagh Fraser',
  'Jack Purvis',
  'Alex McCrindle',
  'Eddie Byrne',
  'Drewe Henley',
  'Denis Lawson',
  'Garrick Hagon',
  'Jack Klaff',
  'William Hootkins',
  'Angus MacInnes',
  'Jeremy Sinden',
  'Graham Ashley',
  'Don Henderson',
  'Richard LeParmentier',
  'Leslie Schofield',
  'Michael Leader',
  'David Ankrum',
  'Scott Beach',
  'Show All…',
  'Lightning Bear',
  'Jon Berg',
  'Doug Beswick',
  'Paul Blake',
  'Janice Burchette',
  'Ted Burnett',
  'John Chapman',
  'Gilda Cohen',
  'Tim Condren',
  'Barry Copping',
  'Alfie Curtis',
  'Robert Davies',
  'Maria De Aragon',
  'Robert A. Denham',
  'Frazer Diamond',
  'Peter Diamond',
  'Warwick Diamond',
  'Sadie Eden',
  'Kim Falkinburg',
 

In [8]:
def scrape_imdb_movie_reviews(movie_id):

    # Specify the path to chromedriver
    service = Service(executable_path='/home/james/.wdm/drivers/chromedriver/linux64/121.0.6167.85/chromedriver-linux64/chromedriver')
    
    # Create ChromeOptions object
    chrome_options = webdriver.ChromeOptions()
    
    # Disable popups
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')  # Disable GPU accelerati

    # Initialize the WebDriver with the Service object
    driver = webdriver.Chrome(service=service,options = chrome_options)

    # url = 'https://www.imdb.com/title/tt'+movie_id+'/reviews'
    # print(url)
    # Open a webpage
    driver.get('https://www.imdb.com/title/tt'+movie_id+'/reviews')

    
    # Wait for the page to load (optional, could use more sophisticated waits)
    time.sleep(5)

    # Wait for the cookies consent button to be clickable, and then click it
    wait = WebDriverWait(driver, 30)
    cookies_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Accept"]')))
    cookies_button.click()

    try:
        max_count = 100
        count = 0
        
        while count < max_count:  # Keep clicking the "Load More" button until it's no longer found

            wait = WebDriverWait(driver, 30)
            load_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Load More"]')))
            load_button.click()
            
            count += 1

            elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'lister-item-content')]")


        #NOTE, webspider not handling submodularisation
        #--------------------------------------------------------------------------------------------------------
        review = {}
        
        for element in elements:
            # print(element)
            #return the individual reviews with their respective ratings
            edivs = element.find_elements(By.TAG_NAME, 'div')
            title_div = element.find_elements(By.TAG_NAME, 'a')

            review = {'ratings':None,'check':''}
            review['title'] = title_div[0].text
            review['movie_id'] = movie_id

            # Step 3: Retrieve and print the text for each child element
            details = {0:'ratings',1:'author',2:'content',3:'check'}
            count = 0

            for div in edivs:
                # print(div.get_attribute('outerHTML'))
                if count in [0,1,2,3]:
                    #possible that there is no rating so check forms.
                    if count == 0:
                        #"10/10"
                        try:
    
                            int(div.text.split('/')[0])
                            review[details[count]] = div.text
                        
                        except:
                            
                            print('review has no rating, skipping')
                            break
                            
                    elif count == 1:
                        data = div.text.split()
                        
                        # print(data)
                        if len(data) >= 1:
                            review['author'] = data[0]
                        if len(data) >= 2:
                            review['date'] = ' '.join(data[1:])
                    else:
    
                        review[details[count]] = div.text
    
                    count +=1
    
            if review['check'] == '':
                #no actual review in these cases.
                review['content'] = None
                
            del review['check']
            # print(review)
            if review['ratings'] != None:
                yield review


    except Exception as e:
        print("All content loaded or button not found.")
    
    
    finally:

    # Close the browser
        driver.quit()

In [9]:
# mix of imdb and tmdb
def search_movie_by_name(movie_name, api_key=TMDB_API_KEY):
    """Searches for a movie by name on TMDb, prints the first result, and fetches reviews including ratings."""
    # Search for the movie

    movie = {'title':None,'release_date':None,'genre':'','director':'','actors':[],'ratings':0,'votes':0,'overview':None,'id':None, 'reviews' : []}
    ia = Cinemagoer()
    # ia = IMDb()
    movies = ia.search_movie(movie_name)
    if movies:
    # Get the first movie from the list
        
        mov = movies[0]
        movkeys = mov.keys()
        print(movkeys)
        # Retrieve detailed information about the movie

        movie['title'] = movie_name
        ia.update(mov)
        if 'genre' in movkeys:
            movie['genre'] = ', '.join(mov['genres'])
        if 'year' in movkeys:
            movie['release date'] = datetime(mov['year'],1,1)
        if 'directors' in movkeys:
            movie['director'] = ', '.join([director['name'] for director in mov['directors']])
        #temporary, even if empty string.
        movie['director_id'] = id(movie['director'])
            
        if 'actors' in movkeys:
            movie['actors'] = [cast['name'] for cast in mov['cast']][:5]
        if 'plot' in movkeys:        
            movie['story'] = mov['plot'][0]
        
        if 'ratings' in movkeys and 'votes' in movkeys:
            
            movie['ratings'] = mov.get('rating')
            movie['votes'] = mov.get('votes')

        movie['id'] = mov['imdbID']
        return movie
    #fill in the details

    else:
        print("No movies found.")

In [10]:
#search_movie_by_name('Avengers: Endgame')

In [11]:
def imdb_scrape_function(movie_name):

    movie = search_movie_by_name(movie_name)
    movie['reviews'] = [rv for rv in scrape_imdb_movie_reviews(movie['id'])]
    return movie

In [12]:
def letterboxd_scrape_function(movie_name):
    #preprocess
    title_translator = str.maketrans('', '', string.punctuation)
    clean_name = movie_name.translate(title_translator)
    # print('clean name = ',clean_name)
    #get movie info, review pages to go through added.
    movie = scrape_letterboxd_movie(clean_name,2)
    #clean up info for db.
    movie = process_letterboxd_movie(movie)
    return movie

In [13]:
movies_q = Queue()

def run_spider(spider):
    while not FileManager.is_empty('todo.txt'):
        task = spider.get_from_file('todo.txt')
        if task:
            #returns review
            movie_info = spider.scrape(task)  # Assuming this method now handles the scraping logic
            #check we actually have a movie worth dishing out????
            if movie_info['reviews']:
                movies_q.enqueue(movie_info)
                spider.push_to_file(task, 'done.txt')
        else:
            break

In [14]:
!cat done.txt

Aquaman
The Fate of the Furious
Avatar: The Way of Water
Top Gun: Maverick
Star Wars: The Force Awakens
The Lion King
Titanic
Jurassic World
The Avengers
Iron Man 3
Avengers: Endgame
Avengers: Infinity War
Harry Potter and the Deathly Hallows: Part 2
Black Panther
Frozen II
The Super Mario Bros. Movie
Barbie
Minions
Jurassic World: Fallen Kingdom
Spider-Man: No Way Home
Beauty and the Beast
Star Wars: The Last Jedi
Frozen
Avatar
Captain America: Civil War
Skyfall
Incredibles 2
Avengers: Age of Ultron
Furious 7


In [15]:
def generate_users(cc,count):

    for i in range(0,count):
        yield User(Faker(random.choice(cc)).name())


# for us in generate_users(["ar_AA"],1):
#     print(us.name())

In [16]:
# TODO content filter
# 
# genre and country,.
# reverse the matching, not most suited, not least suited, 

In [17]:
def main():
    
    db.drop_tables('letterboxd')
    db.create_db('letterboxd')
    
    #create a set of movie_names
    movie_names = list({mn for mn in fetch_movie_names(total_movies = TOTAL_MOVIES)})
    print(movie_names)
    #clean for suitability... to letterboxd scrape.
    
    fm = FileManager()
    fm.write_file('todo.txt')
    fm.write_file('done.txt')

    for movie_name in movie_names:

        fm.enqueue_to_file(movie_name,'todo.txt')
    

    country_codes = [
        "ar_AA", "az_AZ", "bg_BG", "bs_BA", "cs_CZ", "de_DE", "dk_DK", "el_GR",
        "en_AU", "en_CA", "en_GB", "en_IN", "en_NZ", "en_US", "es_ES", "es_MX",
        "et_EE", "fa_IR", "fi_FI", "fr_FR", "hi_IN", "hr_HR", "hu_HU", "hy_AM",
        "id_ID", "it_IT", "ja_JP", "ka_GE", "ko_KR", "lt_LT", "lv_LV", "ne_NP", 
        "nl_NL", "no_NO", "pl_PL", "pt_BR", "pt_PT", "ro_RO", "ru_RU", "sk_SK",
        "sl_SI", "sv_SE", "tr_TR", "uk_UA", "zh_CN", "zh_TW"
    ]
    
    users = [user for user in generate_users(country_codes,TOTAL_USERS)]
    
    #real work, get the movies themselves and reviews.
    spiders = [WebSpider(file_manager=FileManager, scrape_function=letterboxd_scrape_function, name=f"Spider-{i}") for i in range(5)]
    
    threads = [threading.Thread(target=run_spider, args=(spider,)) for spider in spiders]
    
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()
    
    import copy
    
    copy_mq = copy.deepcopy(movies_q)
    user_q = Queue(users)
    leave_q = Queue([])
    
    
    while not movies_q.is_empty():
        mov = movies_q.dequeue()
        
        mov_rq = Queue(mov['reviews'])
        
        #shuffle users before handing out
        random.shuffle(user_q.items)
        # print(len(user_q.items))
        while not user_q.is_empty():
            # if there is anything to hand out.
            user = user_q.dequeue()
            # print(user)
            #we need to hand out to them.....
            if not mov_rq.is_empty():
                review = mov_rq.dequeue()
                review['user_id'] = user.user_id
                user.add_review(review)
                # print(user.name,'has new review',review)
            leave_q.enqueue(user)
            # print(len(user_q))
            # print(len(leave_q))

        #done for that subqueue
        user_q = leave_q
        leave_q = Queue([])

    # print(user_q.items[:10])
    # print(copy_mq.is_empty())

    #now do the DB

    while not user_q.is_empty():
        user = user_q.dequeue()
        db.add_user(user,'letterboxd')
        # print(user.reviews)
        for rv in user.reviews:
            db.add_movie_review(rv,'letterboxd')

    
    while not copy_mq.is_empty():

        movie = copy_mq.dequeue()
        # print(movie['release date'])
        db.add_movie(movie,'letterboxd')    

In [18]:
main()

SQLite database and table created locally.
['Frozen', 'Beauty and the Beast', 'The Avengers', 'The Super Mario Bros. Movie', 'Black Panther', 'Jurassic World: Fallen Kingdom', 'Avatar', 'Jurassic World', 'Avengers: Infinity War', 'Star Wars: The Force Awakens', 'Frozen II', 'Barbie', 'Aquaman', 'Furious 7', 'Avengers: Endgame', 'Incredibles 2', 'Harry Potter and the Deathly Hallows: Part 2', 'The Lion King', 'Iron Man 3', 'Spider-Man: No Way Home', 'Spider-Man: Far From Home', 'The Fate of the Furious', 'Skyfall', 'Avengers: Age of Ultron', 'Captain America: Civil War', 'Avatar: The Way of Water', 'Minions', 'Top Gun: Maverick', 'Titanic', 'Star Wars: The Last Jedi']
File 'todo.txt' created successfully
File 'done.txt' created successfully
Item 'Frozen' successfully enqueued to todo.txt
Item 'Beauty and the Beast' successfully enqueued to todo.txt
Item 'The Avengers' successfully enqueued to todo.txt
Item 'The Super Mario Bros. Movie' successfully enqueued to todo.txt
Item 'Black Panth


KeyboardInterrupt



In [ ]:
print(db.display_size('letterboxd'))

In [ ]:
print(db.display_data('letterboxd'))